Request URL:
https://www.ssfshop.com/public/search/popup/searchGoodList?serviceType=SRC&keyword=%EC%9E%A0%EC%98%B7&orderView=SORT_WEIGHT&pageNo=2&_csrf=8fc51c99-c5be-4fe7-aabb-ef4374c80066&tab=GOD&schType=ALL
Request Method:
GET
Status Code:
200 OK
Remote Address:
45.60.36.250:443
Referrer Policy:
strict-origin-when-cross-origin

payload

serviceType=SRC&keyword=%EC%9E%A0%EC%98%B7&orderView=SORT_WEIGHT&pageNo=2&_csrf=8fc51c99-c5be-4fe7-aabb-ef4374c80066&tab=GOD&schType=ALL

다음은 응답의 일부 입니다.
```
<li class="god-item" view-godno="GR3L24121260828">
								<a href="/PALAN/GR3L24121260828/good?utag=ref_sch:%EC%9E%A0%EC%98%B7$set:2$$dpos:2">
											<div class="god-img">
												</div>

											<div class="god-info">
												<span class="brand">
														PALAN</span>
												<span class="name">
													[2PACK] 밀크 순면 커플 파자마 <em>잠옷</em>세트</span>
												<span class="price">
													<!-- 정소가 노출 로직 추가 : 실소가 > 정소가 노출 > 할인율 -->
													<del><span class="wa_hidden">원가</span>133,800<span class="wa_hidden">원</span></del>
																<em class="sale"><span class="wa_hidden">할인율</span>59%</em>
															<span class="wa_hidden">판매가</span>54,900<span class="wa_hidden">원</span>&nbsp;
														</span>
												<span class="tags">
															<span class="tag">
																	<span class="txt" style="color:#767676;">페이백</span>
																	<span class="bg" style="background:#EFEFEF;"></span>
																</span>
															</span>
													<span class="score">
														<span class="heart"><i aria-label="like"></i><em>1</em></span>
														</span>
												</div>
										</a>
							</li>
```
해당 데이터를 수집하는 코드 작성


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL과 기본 요청 파라미터 정의
url = "https://www.ssfshop.com/public/search/popup/searchGoodList"
params = {
    "serviceType": "SRC",
    "keyword": "잠옷",
    "orderView": "SORT_WEIGHT",
    "pageNo": 1,  # 시작 페이지
    "_csrf": "8fc51c99-c5be-4fe7-aabb-ef4374c80066",
    "tab": "GOD",
    "schType": "ALL"
}

def fetch_page_data(page_no):
    """Fetch data for a specific page"""
    params["pageNo"] = page_no
    response = requests.get(url, params=params)
    if response.status_code != 200:
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.find_all('li', class_='god-item')

    if not items:  # 응답에 데이터가 없으면 종료
        return None

    data = []
    for item in items:
        godno = item.get('view-godno')  # 상품 고유 번호
        link = item.find('a')['href']
        image_tag = item.find('img')
        img_url = image_tag['src'] if image_tag else None

        brand = item.find('span', class_='brand').text.strip() if item.find('span', class_='brand') else None
        name = item.find('span', class_='name').text.strip() if item.find('span', class_='name') else None

        price_tag = item.find('span', class_='price')
        original_price = price_tag.find('del').text.strip() if price_tag and price_tag.find('del') else None
        sale_price = price_tag.find('span', class_='wa_hidden', text='판매가').find_next('span').text.strip() if price_tag else None

        sale_rate = price_tag.find('em', class_='sale').text.strip() if price_tag and price_tag.find('em', class_='sale') else None

        tags = [tag.find('span', class_='txt').text.strip() for tag in item.find_all('span', class_='tag')] if item.find_all('span', class_='tag') else []

        score_tag = item.find('span', class_='score')
        rating = score_tag.find('span', class_='point').find('em').text.strip() if score_tag and score_tag.find('span', class_='point') else None
        reviews = score_tag.find('span', class_='review').find('em').text.strip() if score_tag and score_tag.find('span', class_='review') else None
        likes = score_tag.find('span', class_='heart').find('em').text.strip() if score_tag and score_tag.find('span', class_='heart') else None

        # 데이터 저장
        data.append({
            "godno": godno,
            "link": link,
            "img_url": img_url,
            "brand": brand,
            "name": name,
            "original_price": original_price,
            "sale_price": sale_price,
            "sale_rate": sale_rate,
            "tags": tags,
            "rating": rating,
            "reviews": reviews,
            "likes": likes
        })

    return data

all_data = []
current_page = 1

while True:
    print(f"Fetching page {current_page}...")
    page_data = fetch_page_data(current_page)
    if not page_data:  # 데이터가 없으면 중지
        break
    all_data.extend(page_data)
    current_page += 1

# 수집된 데이터 확인 및 데이터프레임으로 변환
df = pd.DataFrame(all_data)
print(f"Total pages fetched: {current_page - 1}")
print(df)


In [ ]:
df = pd.DataFrame(all_data)
df

In [ ]:
df